In [ ]:
from pipython import GCSDevice
from pipython import datarectools, pitools
from IPython.display import clear_output
import os
import time
from datetime import datetime
import numpy as np
import pandas as pd
import threading
import subprocess
import matplotlib.pyplot as plt
import matplotlib.animation as ani
STAGES = ['P-611.3S','P-611.3S','P-611.3S']
REFMODES = None
pidevice = GCSDevice()
#pidevice.InterfaceSetupDlg()
drec = datarectools.Datarecorder(pidevice)

In [ ]:
def loop(r,t,d,v): #r-record sample rate/s;t=record time;d=displacement;v=velocity;;;; t should be at minimum d/v
    pidevice = GCSDevice()
    pidevice.ConnectUSB(serialnum='0119052197')
    drec = datarectools.Datarecorder(pidevice)
    #drec.samplefrequ = 1000
    #drec.samplerate = 5
    drec.sampletime = r
    #print('data recorder rate: {:d} servo cycles'.format(drec.samplerate))
    #print('data recorder rate: {:.g} seconds'.format(drec.sampletime))
    #print('data recorder rate: {:.2f} Hertz'.format(drec.samplefrequ))
    drec.options = [datarectools.RecordOptions.ACTUAL_POSITION_2,
                    datarectools.RecordOptions.SENSOR_NORMALIZED_17]
    drec.sources = [3,4]
    #print(drec.rectime())
    drec.rectime = t
    drec.trigsources = datarectools.TriggerSources.POSITION_CHANGING_COMMAND_1
    drec.arm()
    pidevice.VEL(3,v)
    #pidevice.MOV(3,48)
    pidevice.MVR(3,d)
    drec.wait()
    header, data = drec.read()
    timescale = [header['SAMPLE_TIME'] * i for i in range(len(data[0]))]
    npdataL1 = np.array([data[0],data[1]])
    nptime = np.array(timescale)
    npdataL1 = np.concatenate((nptime.reshape(1, -1),npdataL1,), axis=0)
    dataset = pd.DataFrame(npdataL1.T)
    return dataset

In [ ]:
#General position and force #### Don't use yet. I need to fix definitions
speed(3,10)
move(3,0)
#move(1,40)
f_0 = force()
k_0 = disp()
print(position(),force())

In [ ]:
#Approach ##### Not very reliable!I need to fix definitions based off of new info
speed(3,1)
max_approach_len = 50 #um
step_size = 0.2 #um
max_force = 0.005 #N

for i in range(1000):
    relmove(3,step_size)
    n = force()-f_0
    print(disp(),n)
    if n > max_force or n == max_force:
        relmove(3,-2*step_size)
        n = force()-f_0
        k_0 = disp()
        print("approach complete: ", k_0,"um ", n,"N")
        break
    if disp() > max_approach_len or disp() == max_approach_len:
        print("Maximum approach distance limit")
        break


In [ ]:
# multiple load-displacement (each if statement is a different action)
folder_name = 'Polymer_Eraser02142025'
# Create the folder if it doesn't already exist
os.makedirs(folder_name, exist_ok=True)
r = 0.2
v = 0.1
for i in range(9):
    if i == 0:
        # Generate DataFrame
        df = loop(r,10/v,10,v)   #r-record sample rate/s;t=record time;d=displacement;v=velocity;;;; t should be at minimum d/v 
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 1:
        # Generate DataFrame
        df = loop(r,10/v,0,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 2:
        # Generate DataFrame
        df = loop(r,10/v,-10,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 3:
        # Generate DataFrame
        df = loop(r,20/v,20,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 4:
        # Generate DataFrame
        df = loop(r,20/v,0,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 5:
        # Generate DataFrame
        df = loop(r,20/v,-20,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 6:
        # Generate DataFrame
        df = loop(r,40/v,40,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 7:
        # Generate DataFrame
        df = loop(r,40/v,0,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")
    if i == 8:
        # Generate DataFrame
        df = loop(r,40/v,-40,v)    
        # Generate the file name dynamically based on the index
        file_name = f"{folder_name}/name_{i}.csv"
        # Export DataFrame to CSV inside the new folder
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")

In [ ]:
# single movement load-displacement (can't apply multiple move commands)
pidevice = GCSDevice()
pidevice.ConnectUSB(serialnum='0119052197')
drec = datarectools.Datarecorder(pidevice)
#drec.samplefrequ = 1000
#drec.samplerate = 5
drec.sampletime = 0.2
#print('data recorder rate: {:d} servo cycles'.format(drec.samplerate))
print('data recorder rate: {:.g} seconds'.format(drec.sampletime))
#print('data recorder rate: {:.2f} Hertz'.format(drec.samplefrequ))
drec.options = [datarectools.RecordOptions.ACTUAL_POSITION_2,
                datarectools.RecordOptions.SENSOR_NORMALIZED_17]
drec.sources = [3,4]
#print(drec.rectime())
drec.rectime = 305
drec.trigsources = datarectools.TriggerSources.POSITION_CHANGING_COMMAND_1
drec.arm()
pidevice.VEL(3,0.2)
#pidevice.MOV(3,48)
pidevice.MVR(3,60)
drec.wait()
header, data = drec.read()


In [ ]:
#Plot single run above
timescale = [header['SAMPLE_TIME'] * i for i in range(len(data[0]))]
#plt.plot(timescale, data[0], color='red') #time [s] and displacement [um]
#plt.plot(timescale, data[1], color='blue') #time [s] and voltage [V]
plt.plot(data[0],data[1]) #displacement [um] and voltage [V]
plt.xlabel('time (s)')
plt.ylabel(', '.join((header['NAME0'], header['NAME1']))) #change based on which plot
#plt.title('Recorded data over time')
plt.grid(True)
plt.show()

In [ ]:
#Export single run
npdataL1 = np.array([data[0],data[1]])
nptime = np.array(timescale)
npdataL1 = np.concatenate((npdataL1,nptime.reshape(1, -1)), axis=0)
dataset = pd.DataFrame(npdataL1.T)
dataset.to_csv('eraser_U60um.csv', index=False)